In [1]:
pip install fer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 53.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 72.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for ffmpeg: filename=ffmpeg-1.4-py3-none-any.whl size=6083 sha256=c12c0ed56359d2ea3445857f42c665dc70ee92860be560da45f9156b7dbdf5da
  Stored in directory: /root/.cache/pip/wheels/8e/7a/69/cd6aeb83b126a7f04cbe7c9d929028dc52a6e7d525ff56003a
Successfully built ffmpeg


In [2]:
from fer import Video
from fer import FER
import os
import sys
import pandas as pd
from google.colab import drive

drive.mount('/content/6-10')

Mounted at /content/6-10


In [3]:
#cd sample_data
#ls

In [4]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [5]:
# Put in the location of the video file that has to be processed
file_name = "/content/6-10/MyDrive/11-32/c5_2.mp4"
location_videofile = file_name

In [6]:
# Build the Face detection detector
face_detector = FER(mtcnn=True)
# Input the video for processing
input_video = Video(location_videofile)

In [ ]:
# The Analyze() function will run analysis on every frame of the input video.
# It will create a rectangular box around every image and show the emotion values next to that.
# Finally, the method will publish a new video that will have a box around the face of the human with live emotion values.

with tf.device('/device:GPU:0'):
  processing_data = input_video.analyze(face_detector, display=False, frequency=2.5)

INFO:fer:30.00 fps, 9000 frames, 300.05 seconds
INFO:fer:Making directories at output
 19%|█▊        | 1671/9000 [05:37<23:29,  5.20frames/s]

In [ ]:
# We will now convert the analysed information into a dataframe.
# This will help us import the data as a .CSV file to perform analysis over it later
vid_df = input_video.to_pandas(processing_data)
#vid_df = input_video.get_first_face(vid_df)
vid_df = input_video.get_emotions(vid_df)
vid_df.to_csv('/content/6-10/MyDrive/11-32/output/df_c5_2.csv', index=False)
vid_df

In [ ]:
# Plotting the emotions against time in the video
#vid_df = pd.read_csv("/content/6-10/MyDrive/6-10/df_c21.csv")
pltfig = vid_df.plot(figsize=(50, 10), fontsize=16).get_figure()
pltfig.savefig('/content/6-10/MyDrive/11-32/output/emotions_timeline_c5_2.png', dpi=pltfig.dpi)

In [ ]:
# We will now work on the dataframe to extract which emotion was prominent in the video
angry = sum(vid_df.angry)
disgust = sum(vid_df.disgust)
fear = sum(vid_df.fear)
happy = sum(vid_df.happy)
sad = sum(vid_df.sad)
surprise = sum(vid_df.surprise)
neutral = sum(vid_df.neutral)

In [ ]:
emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
emotions_values = [angry, disgust, fear, happy, sad, surprise, neutral]

In [ ]:
score_comparisons = pd.DataFrame(emotions, columns = ['Human Emotions'])
score_comparisons['Emotion Value from the Video'] = emotions_values
score_comparisons.to_csv("/content/6-10/MyDrive/11-32/output/score_comparisons_c5_2", sep=',', index=False)
score_comparisons

In [ ]:
'''from multiprocessing import Process

def print_func(filename = '/content/6-10/MyDrive/6-10/konventiert/c2_1.mp4'):
    # Put in the location of the video file that has to be processed
    file_name = name
    location_videofile = file_name

    # Build the Face detection detector
    face_detector = FER(mtcnn=True)
    # Input the video for processing
    input_video = Video(location_videofile)

    # The Analyze() function will run analysis on every frame of the input video.
    # It will create a rectangular box around every image and show the emotion values next to that.
    # Finally, the method will publish a new video that will have a box around the face of the human with live emotion values.
    with tf.device('/device:GPU:0'):
      string = name + "processing_data"
      processing_data = input_video.analyze(face_detector, display=False, frequency=2)

    # We will now convert the analysed information into a dataframe.
    # This will help us import the data as a .CSV file to perform analysis over it later
    vid_df = input_video.to_pandas(processing_data)
    vid_df = input_video.get_first_face(vid_df)
    vid_df = input_video.get_emotions(vid_df)

    # Plotting the emotions against time in the video
    pltfig = vid_df.plot(figsize=(20, 8), fontsize=16).get_figure()
    fil = 'emotions_timeline'+ name + '.png'
    pltfig.savefig(fil, dpi=pltfig.dpi)

    # We will now work on the dataframe to extract which emotion was prominent in the video
    angry = sum(vid_df.angry)
    disgust = sum(vid_df.disgust)
    fear = sum(vid_df.fear)
    happy = sum(vid_df.happy)
    sad = sum(vid_df.sad)
    surprise = sum(vid_df.surprise)
    neutral = sum(vid_df.neutral)

    emotions = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
    emotions_values = [angry, disgust, fear, happy, sad, surprise, neutral]

    score_comparisons = pd.DataFrame(emotions, columns = ['Human Emotions'])
    score_comparisons['Emotion Value from the Video'] = emotions_values
    fil = "score_comparison"+ name
    score_comparisons.to_csv(fil, sep=',', index=False, encoding='utf-8')
    score_comparisons

In [ ]:
'''if __name__ == "__main__":
    names = [ '/content/6-10/MyDrive/6-10/konventiert/c2_2.mp4', '/content/6-10/MyDrive/6-10/konventiert/c2_3.mp4', '/content/6-10/MyDrive/6-10/konventiert/c2_4.pv4']
    procs = []
    proc = Process(target=print_func)  # instantiating without any argument
    procs.append(proc)
    proc.start()

    # instantiating process with arguments
    for name in names:
        # print(name)
        proc = Process(target=print_func, args=(name,))
        procs.append(proc)
        proc.start()

    # complete the processes
    for proc in procs:
        proc.join()